In [ ]:
import variational_bayes as vb
import numpy as np
from matplotlib import pyplot as plt, rcParams
import scipy.stats
from tqdm import tqdm_notebook
%matplotlib inline
%load_ext cython

rcParams['figure.dpi'] = 144
rcParams['scatter.marker'] = '.'

In [ ]:
np.random.seed(3)
num_groups = 2
num_nodes = 50
order = 2
num_steps = 10 * num_nodes * num_nodes

# Generate group sizes and groups
density = np.random.dirichlet(100 * np.ones(num_groups)) # was 10 when things didn't work
z = np.random.choice(num_groups, num_nodes, p=density)
onehot = np.zeros((num_nodes, num_groups))
onehot[np.arange(num_nodes), z] = 1

# Sample noise precisions for all groups
noise_precision = np.random.gamma(5000, size=num_groups)
# noise_precision = 100

# Sample means and precisions of autoregressive coefficients
adjacency_mean = np.random.normal(0, 1e-2, size=(num_groups, num_groups, order))
adjacency_precision = scipy.stats.wishart.rvs(1e4, np.eye(order), size=(num_groups, num_groups))

# Sample the means and precisions of the bias
bias_mean = np.random.normal(0, 0.1, num_groups)
bias_precision = np.random.gamma(1e4, 1, num_groups)

# Sample the matrix of autoregressive coefficients
cholesky = np.linalg.cholesky(np.linalg.inv(adjacency_precision))
cholesky = cholesky[z[:, None], z[None, :]]
adjacency = adjacency_mean[z[:, None], z[None, :]] + \
    np.einsum('...ij,...j', cholesky, np.random.normal(0, 1, (num_nodes, num_nodes, order)))
    
# Sample the bias
bias = np.random.normal(0, 1, num_nodes) / np.sqrt(bias_precision[z]) + bias_mean[z]

# Construct the coefficients for comparison
coefficients = vb.pack_coefficients(adjacency, bias)
    
# Generate the actual series
x = vb.simulate_series(bias, adjacency, noise_precision[z], num_steps)

In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2)
ax1.plot(x)
ax2.scatter(*adjacency.reshape((-1, 2)).T, 
            c=(z[:, None] * num_groups + z[None, :]).ravel(),
            cmap='Set1')
ax2.set_aspect('equal')

for i in range(num_groups):
    ax3.hist(bias[z == i], range=(np.min(bias), np.max(bias)))

In [ ]:
np.random.seed(15)
model = vb.var_model(x, order, num_groups)

factor_order = ['coefficients', 'noise_precision', 'adjacency_mean', 
                'adjacency_precision', 'bias_mean', 'bias_precision', 'z']
elbos, _ = model.update(None, tqdm_notebook, factor_order, convergence_predicate=vb.ConvergencePredicate(1e-3, 10))
plt.plot(elbos)

In [ ]:
plt.imshow(model['z'].mean[np.argsort(z)], aspect='auto')

In [ ]:
plt.figure()
vb.plot_comparison(model['bias_mean'], bias_mean, marker='.')
plt.title('bias_mean')

In [ ]:
plt.figure()
vb.plot_comparison(model['bias_precision'], bias_precision, marker='.')
plt.title('bias_precision')

In [ ]:
plt.figure()
vb.plot_comparison(model['adjacency_precision'], adjacency_precision, marker='.')
plt.title('adjacency_precision')

In [ ]:
plt.figure()
vb.plot_residuals(model['adjacency_mean'], adjacency_mean, marker='.')
plt.title('adjacency_mean')

plt.figure()
_x, _y = adjacency.reshape((-1, order)).T
plt.scatter(_x, _y, c=(z[:, None] * num_groups + z[None, :]).ravel(),
            cmap='Set1', zorder=9)
_x, _y = model['adjacency_mean'].mean.reshape((-1, 2)).T
_xerr, _yerr = model['adjacency_mean'].std.reshape((-1, 2)).T
plt.errorbar(_x, _y, _yerr, _xerr, color='k', ls='none', zorder=99)
plt.scatter(_x, _y, marker='x', color='k', zorder=99)
plt.gca().set_aspect(1)
plt.title('adjacency_mean')

In [ ]:
plt.figure()
vb.plot_comparison(model['density'], density, marker='.')
plt.title('density')

In [ ]:
q_adjacency = vb.VARAdjacencyDistribution(model['coefficients'])
_xerr, _yerr = q_adjacency.std.reshape(-1, order).T
_x, _y = q_adjacency.mean.reshape((-1, order)).T
plt.errorbar(_x, _y, _yerr, _xerr, ls='none', c='k', alpha=.5)
plt.scatter(_x, _y, c=(z[:, None] * num_groups + z[None, :]).ravel(),
            cmap='Set1', zorder=9)
plt.gca().set_aspect(1)